In [187]:
import numpy as np
import pandas as pd
import tweepy
import json
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
from nltk.corpus import stopwords
sw=stopwords.words("english")

In [168]:
pd.set_option('display.max_colwidth', -1)

# gathering and formating

In [4]:
folder_name ='Supporting Materials'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

wb b is for writing image file

In [5]:
url= ['https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv',
     'https://video.udacity-data.com/topher/2018/November/5bf60c1e_twitter-archive-enhanced-2/twitter-archive-enhanced-2.csv',
     'https://video.udacity-data.com/topher/2018/November/5be5fb7d_tweet-json/tweet-json.txt']
for i in url:
    response = requests.get(i)
    with open(os.path.join(folder_name,i.split('/')[-1]), mode='wb') as f:
        f.write(response.content)

In [6]:
archive_df=pd.read_csv('Supporting Materials/twitter-archive-enhanced-2.csv')

In [5]:
archive_df.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [7]:
image_df = pd.read_csv('Supporting Materials/image-predictions.tsv',sep='\t')

In [7]:
image_df.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [8]:

consumer_key = 'YOUR CONSUMER KEY'
consumer_secret = 'YOUR CONSUMER SECRET'
access_token = 'YOUR ACCESS TOKEN'
access_secret = 'YOUR ACCESS SECRET'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth,wait_on_rate_limit =True,wait_on_rate_limit_notify=True)

In [9]:
id_of_tweet=archive_df['tweet_id'].values

In [ ]:
t0=time.time()
fails=[]
with open('tweet_json.txt','w') as f:
    count=0
    for i in id_of_tweet:
        count+=1
        print(count,':', i)
        try:
            tweet = api.get_status(i,tweet_mode='extended')
            print('success')
            json.dump(tweet._json,f)
            f.write('\n')
        except:
            print('fail')
            fails.append[i]
print(time.time()-t0)
print(fails)

1 : 892420643555336193
fail
2 : 892177421306343426
fail
3 : 891815181378084864
fail
4 : 891689557279858688
fail
5 : 891327558926688256
fail
6 : 891087950875897856
fail
7 : 890971913173991426
fail
8 : 890729181411237888
fail
9 : 890609185150312448
fail
10 : 890240255349198849
fail
11 : 890006608113172480
fail
12 : 889880896479866881
fail
13 : 889665388333682689
fail
14 : 889638837579907072
fail
15 : 889531135344209921
fail
16 : 889278841981685760
fail
17 : 888917238123831296
fail
18 : 888804989199671297
fail
19 : 888554962724278272
fail
20 : 888202515573088257
fail
21 : 888078434458587136
fail
22 : 887705289381826560
fail
23 : 887517139158093824
fail
24 : 887473957103951883
fail
25 : 887343217045368832
fail
26 : 887101392804085760
fail
27 : 886983233522544640
fail
28 : 886736880519319552
fail
29 : 886680336477933568
fail
30 : 886366144734445568
fail
31 : 886267009285017600
fail
32 : 886258384151887873
fail
33 : 886054160059072513
fail
34 : 885984800019947520
fail
35 : 885528943205470208

fail
278 : 840370681858686976
fail
279 : 840268004936019968
fail
280 : 839990271299457024
fail
281 : 839549326359670784
fail
282 : 839290600511926273
fail
283 : 839239871831150596
fail
284 : 838952994649550848
fail
285 : 838921590096166913
fail
286 : 838916489579200512
fail
287 : 838831947270979586
fail
288 : 838561493054533637
fail
289 : 838476387338051585
fail
290 : 838201503651401729
fail
291 : 838150277551247360
fail
292 : 838085839343206401
fail
293 : 838083903487373313
fail
294 : 837820167694528512
fail
295 : 837482249356513284
fail
296 : 837471256429613056
fail
297 : 837366284874571778
fail
298 : 837110210464448512
fail
299 : 837012587749474308
fail
300 : 836989968035819520
fail
301 : 836753516572119041
fail
302 : 836677758902222849
fail
303 : 836648853927522308
fail
304 : 836397794269200385
fail
305 : 836380477523124226
fail
306 : 836260088725786625
fail
307 : 836001077879255040
fail
308 : 835685285446955009
fail
309 : 835574547218894849
fail
310 : 835536468978302976
fail
311 :

fail
552 : 804475857670639616
fail
553 : 804413760345620481
fail
554 : 804026241225523202
fail
555 : 803773340896923648
fail
556 : 803692223237865472
fail
557 : 803638050916102144
fail
558 : 803380650405482500
fail
559 : 803321560782307329
fail
560 : 803276597545603072
fail
561 : 802952499103731712
fail
562 : 802624713319034886
fail
563 : 802600418706604034
fail
564 : 802572683846291456
fail
565 : 802323869084381190
fail
566 : 802265048156610565
fail
567 : 802247111496568832
fail
568 : 802239329049477120
fail
569 : 802185808107208704
fail
570 : 801958328846974976
fail
571 : 801854953262350336
fail
572 : 801538201127157760
fail
573 : 801285448605831168
fail
574 : 801167903437357056
fail
575 : 801127390143516673
fail
576 : 801115127852503040
fail
577 : 800859414831898624
fail
578 : 800855607700029440
fail
579 : 800751577355128832
fail
580 : 800513324630806528
fail
581 : 800459316964663297
fail
582 : 800443802682937345
fail
583 : 800388270626521089
fail
584 : 800188575492947969
fail
585 :

fail
826 : 769695466921623552
fail
827 : 769335591808995329
fail
828 : 769212283578875904
fail
829 : 768970937022709760
fail
830 : 768909767477751808
fail
831 : 768855141948723200
fail
832 : 768609597686943744
fail
833 : 768596291618299904
fail
834 : 768554158521745409
fail
835 : 768473857036525572
fail
836 : 768193404517830656
fail
837 : 767884188863397888
fail
838 : 767754930266464257
fail
839 : 767500508068192258
fail
840 : 767191397493538821
fail
841 : 767122157629476866
fail
842 : 766864461642756096
fail
843 : 766793450729734144
fail
844 : 766714921925144576
fail
845 : 766693177336135680
fail
846 : 766423258543644672
fail
847 : 766313316352462849
fail
848 : 766078092750233600
fail
849 : 766069199026450432
fail
850 : 766008592277377025
fail
851 : 765719909049503744
fail
852 : 765669560888528897
fail
853 : 765395769549590528
fail
854 : 765371061932261376
fail
855 : 765222098633691136
fail
856 : 764857477905154048
fail
857 : 764259802650378240
fail
858 : 763956972077010945
fail
859 :

fail
1097 : 736365877722001409
fail
1098 : 736225175608430592
fail
1099 : 736010884653420544
fail
1100 : 735991953473572864
fail
1101 : 735648611367784448
fail
1102 : 735635087207878657
fail
1103 : 735274964362878976
fail
1104 : 735256018284875776
fail
1105 : 735137028879360001
fail
1106 : 734912297295085568
fail
1107 : 734787690684657664
fail
1108 : 734776360183431168
fail
1109 : 734559631394082816
fail
1110 : 733828123016450049
fail
1111 : 733822306246479872
fail
1112 : 733482008106668032
fail
1113 : 733460102733135873
fail
1114 : 733109485275860992
fail
1115 : 732732193018155009
fail
1116 : 732726085725589504
fail
1117 : 732585889486888962
fail
1118 : 732375214819057664
fail
1119 : 732005617171337216
fail
1120 : 731285275100512256
fail
1121 : 731156023742988288
fail
1122 : 730924654643314689
fail
1123 : 730573383004487680
fail
1124 : 730427201120833536
fail
1125 : 730211855403241472
fail
1126 : 730196704625098752
fail
1127 : 729854734790754305
fail
1128 : 729838605770891264
fail
112

fail
1362 : 703079050210877440
fail
1363 : 703041949650034688
fail
1364 : 702932127499816960
fail
1365 : 702899151802126337
fail
1366 : 702684942141153280
fail
1367 : 702671118226825216
fail
1368 : 702598099714314240
fail
1369 : 702539513671897089
fail
1370 : 702332542343577600
fail
1371 : 702321140488925184
fail
1372 : 702276748847800320
fail
1373 : 702217446468493312
fail
1374 : 701981390485725185
fail
1375 : 701952816642965504
fail
1376 : 701889187134500865
fail
1377 : 701805642395348998
fail
1378 : 701601587219795968
fail
1379 : 701570477911896070
fail
1380 : 701545186879471618
fail
1381 : 701214700881756160
fail
1382 : 700890391244103680
fail
1383 : 700864154249383937
fail
1384 : 700847567345688576
fail
1385 : 700796979434098688
fail
1386 : 700747788515020802
fail
1387 : 700518061187723268
fail
1388 : 700505138482569216
fail
1389 : 700462010979500032
fail
1390 : 700167517596164096
fail
1391 : 700151421916807169
fail
1392 : 700143752053182464
fail
1393 : 700062718104104960
fail
139

fail
1627 : 684800227459624960
fail
1628 : 684594889858887680
fail
1629 : 684588130326986752
fail
1630 : 684567543613382656
fail
1631 : 684538444857667585
fail
1632 : 684481074559381504
fail
1633 : 684460069371654144
fail
1634 : 684241637099323392
fail
1635 : 684225744407494656
fail
1636 : 684222868335505415
fail
1637 : 684200372118904832
fail
1638 : 684195085588783105
fail
1639 : 684188786104872960
fail
1640 : 684177701129875456
fail
1641 : 684147889187209216
fail
1642 : 684122891630342144
fail
1643 : 684097758874210310
fail
1644 : 683857920510050305
fail
1645 : 683852578183077888
fail
1646 : 683849932751646720
fail
1647 : 683834909291606017
fail
1648 : 683828599284170753
fail
1649 : 683773439333797890
fail
1650 : 683742671509258241
fail
1651 : 683515932363329536
fail
1652 : 683498322573824003
fail
1653 : 683481228088049664
fail
1654 : 683462770029932544
fail
1655 : 683449695444799489
fail
1656 : 683391852557561860
fail
1657 : 683357973142474752
fail
1658 : 683142553609318400
fail
165

fail
1892 : 674764817387900928
fail
1893 : 674754018082705410
fail
1894 : 674752233200820224
fail
1895 : 674743008475090944
fail
1896 : 674742531037511680
fail
1897 : 674739953134403584
fail
1898 : 674737130913071104
fail
1899 : 674690135443775488
fail
1900 : 674670581682434048
fail
1901 : 674664755118911488
fail
1902 : 674646392044941312
fail
1903 : 674644256330530816
fail
1904 : 674638615994089473
fail
1905 : 674632714662858753
fail
1906 : 674606911342424069
fail
1907 : 674468880899788800
fail
1908 : 674447403907457024
fail
1909 : 674436901579923456
fail
1910 : 674422304705744896
fail
1911 : 674416750885273600
fail
1912 : 674410619106390016
fail
1913 : 674394782723014656
fail
1914 : 674372068062928900
fail
1915 : 674330906434379776
fail
1916 : 674318007229923329
fail
1917 : 674307341513269249
fail
1918 : 674291837063053312
fail
1919 : 674271431610523648
fail
1920 : 674269164442398721
fail
1921 : 674265582246694913
fail
1922 : 674262580978937856
fail
1923 : 674255168825880576
fail
192

fail
2158 : 669583744538451968
fail
2159 : 669573570759163904
fail
2160 : 669571471778410496
fail
2161 : 669567591774625800
fail
2162 : 669564461267722241
fail
2163 : 669393256313184256
fail
2164 : 669375718304980992
fail
2165 : 669371483794317312
fail
2166 : 669367896104181761
fail
2167 : 669363888236994561
fail
2168 : 669359674819481600
fail
2169 : 669354382627049472
fail
2170 : 669353438988365824
fail
2171 : 669351434509529089
fail
2172 : 669328503091937280
fail
2173 : 669327207240699904
fail
2174 : 669324657376567296
fail
2175 : 669216679721873412
fail
2176 : 669214165781868544
fail
2177 : 669203728096960512
fail
2178 : 669037058363662336
fail
2179 : 669015743032369152
fail
2180 : 669006782128353280
fail
2181 : 669000397445533696
fail
2182 : 668994913074286592
fail
2183 : 668992363537309700
fail
2184 : 668989615043424256
fail
2185 : 668988183816871936
fail
2186 : 668986018524233728
fail
2187 : 668981893510119424
fail
2188 : 668979806671884288
fail
2189 : 668975677807423489
fail
219

In [23]:
tweetlist=[]
with open('Supporting Materials/tweet-json.txt') as f:
    for line in f:
        data=json.loads(line)
        tweet_id=data['id']
        retweet_count=data['retweet_count']
        favourite_count=data['favorite_count']
        tweetdict={'tweet_id':tweet_id,
                  'retweet_count':retweet_count,
                  'favourite_count':favourite_count}
        tweetlist.append(tweetdict)

In [25]:
json_df=pd.DataFrame(tweetlist)

In [26]:
json_df.head()

,tweet_id,retweet_count,favourite_count
0,892420643555336193,8853,39467
1,892177421306343426,6514,33819
2,891815181378084864,4328,25461
3,891689557279858688,8964,42908
4,891327558926688256,9774,41048


In [617]:
# make copies
adf = archive_df.copy(deep=True)
idf = image_df.copy(deep=True)
jdf = json_df.copy(deep=True)

In [172]:
adf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

# wrangling

In [618]:
for i ,z in enumerate(adf):
    print (i,z)

0 tweet_id
1 in_reply_to_status_id
2 in_reply_to_user_id
3 timestamp
4 source
5 text
6 retweeted_status_id
7 retweeted_status_user_id
8 retweeted_status_timestamp
9 expanded_urls
10 rating_numerator
11 rating_denominator
12 name
13 doggo
14 floofer
15 pupper
16 puppo


> drop duplicates

In [619]:
adf=adf[adf['retweeted_status_id'].isna()]

> drop columns

In [620]:
adf.columns[np.r_[1:3,4,6:9,10:12]]

Index(['in_reply_to_status_id', 'in_reply_to_user_id', 'source',
       'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'rating_numerator', 'rating_denominator'],
      dtype='object')

In [621]:
adf.drop(adf.columns[np.r_[1:3,4,6:9,10:12]],axis=1,inplace=True)

In [622]:
adf

,tweet_id,timestamp,text,expanded_urls,name,doggo,floofer,pupper,puppo
0,892420643555336193,2017-08-01 16:23:56 +0000,This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,https://twitter.com/dog_rates/status/892420643555336193/photo/1,Phineas,None,None,None,None
1,892177421306343426,2017-08-01 00:17:27 +0000,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",https://twitter.com/dog_rates/status/892177421306343426/photo/1,Tilly,None,None,None,None
2,891815181378084864,2017-07-31 00:18:03 +0000,This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,https://twitter.com/dog_rates/status/891815181378084864/photo/1,Archie,None,None,None,None
3,891689557279858688,2017-07-30 15:58:51 +0000,This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,https://twitter.com/dog_rates/status/891689557279858688/photo/1,Darla,None,None,None,None
4,891327558926688256,2017-07-29 16:00:24 +0000,"This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f","https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",Franklin,None,None,None,None
...,...,...,...,...,...,...,...,...,...
2351,666049248165822465,2015-11-16 00:24:50 +0000,Here we have a 1949 1st generation vulpix. Enjoys sweat tea and Fox News. Cannot be phased. 5/10 https://t.co/4B7cOc1EDq,https://twitter.com/dog_rates/status/666049248165822465/photo/1,None,None,None,None,None
2352,666044226329800704,2015-11-16 00:04:52 +0000,This is a purebred Piers Morgan. Loves to Netflix and chill. Always looks like he forgot to unplug the iron. 6/10 https://t.co/DWnyCjf2mx,https://twitter.com/dog_rates/status/666044226329800704/photo/1,a,None,None,None,None
2353,666033412701032449,2015-11-15 23:21:54 +0000,Here is a very happy pup. Big fan of well-maintained decks. Just look at that tongue. 9/10 would cuddle af https://t.co/y671yMhoiR,https://twitter.com/dog_rates/status/666033412701032449/photo/1,a,None,None,None,None
2354,666029285002620928,2015-11-15 23:05:30 +0000,This is a western brown Mitsubishi terrier. Upset about leaf. Actually 2 dogs here. 7/10 would walk the shit out of https://t.co/r7mOb2m0UI,https://twitter.com/dog_rates/status/666029285002620928/photo/1,a,None,None,None,None


In [623]:
adf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 9 columns):
tweet_id         2175 non-null int64
timestamp        2175 non-null object
text             2175 non-null object
expanded_urls    2117 non-null object
name             2175 non-null object
doggo            2175 non-null object
floofer          2175 non-null object
pupper           2175 non-null object
puppo            2175 non-null object
dtypes: int64(1), object(8)
memory usage: 169.9+ KB


> change datatypes

In [624]:
adf['timestamp']=adf['timestamp'].astype('datetime64')

>reextract names

In [651]:
names=adf.copy()

In [652]:
names['name'].value_counts()

None       680
a          55 
Charlie    11 
Lucy       11 
Cooper     10 
           .. 
River      1  
Bayley     1  
Jamesy     1  
Mac        1  
Pherb      1  
Name: name, Length: 956, dtype: int64

In [653]:
testdf=names['text'].str.extract(r'\w?([Tt]his is [A-Z][A-Za-z]+)|\w?(hello to [A-Z][A-Za-z]+)|\w?(Meet [A-Z][A-Za-z]+)|\w?([Tt]hat is [A-Z][A-Za-z]+)|\w?([Tt]his is [A-Z].[A-Z][A-Za-z]+)|\w?(named [A-Z][A-Za-z]+)')
testdf

,0,1,2,3,4,5
0,This is Phineas,NaN,NaN,NaN,NaN,NaN
1,This is Tilly,NaN,NaN,NaN,NaN,NaN
2,This is Archie,NaN,NaN,NaN,NaN,NaN
3,This is Darla,NaN,NaN,NaN,NaN,NaN
4,This is Franklin,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2351,NaN,NaN,NaN,NaN,NaN,NaN
2352,NaN,NaN,NaN,NaN,NaN,NaN
2353,NaN,NaN,NaN,NaN,NaN,NaN
2354,NaN,NaN,NaN,NaN,NaN,NaN


In [654]:
cols=[0,1,2,3,4,5]
for i in cols:
    testdf[0]= testdf[0].fillna(testdf[i])

In [655]:
testdf[0].value_counts()

This is Lucy         10
This is Cooper       9 
This is Charlie      8 
This is Oliver       8 
This is Penny        8 
                    .. 
This is Obi          1 
Meet Sojourner       1 
Meet Griswold        1 
This is Tove         1 
This is Bookstore    1 
Name: 0, Length: 1065, dtype: int64

In [656]:
names['name']=testdf[0].str.split(' ').str[-1]

In [665]:
names[['text','name']].sample(10)

,text,name
816,This is Winston. His tongue has gone rogue. Doing him quite a frighten. 10/10 hang in there Winston https://t.co/d0QEbp78Yi,Winston
326,Meet Charlie. She asked u to change the channel to Animal Planet at least 6 times. Now taking matters into her own paws. 13/10 assertive af https://t.co/WTzhtfevKY,Charlie
1902,When you see sophomores in high school driving. 11/10 https://t.co/m6aC8d1Kzp,NaN
1493,"""Hello yes could I get one pupper to go please thank you""\nBoth 13/10 https://t.co/kYWcXbluUu",NaN
471,This is Binky. She appears to be rather h*ckin cozy. Nifty leg cross as well. 12/10 would snug well https://t.co/WFt82XLyEF,Binky
1464,"This pupper only appears through the hole of a Funyun. Much like Phineas, this one is also mysterious af. 10/10 https://t.co/SQsEBWxPyG",NaN
1870,holy shit 12/10 https://t.co/p6O8X93bTQ,NaN
1510,This is Bailey. She likes flowers. 12/10 https://t.co/YBENhr24FV,Bailey
1659,aahhhhkslaldhwnxmzbbs 12/10 for being da smooshiest https://t.co/UOPdXmUz4H,NaN
375,Beebop and Doobert should start a band 12/10 would listen,NaN


In [658]:
adf['name']=names['name']

In [667]:
adf

,tweet_id,timestamp,text,expanded_urls,name,doggo,floofer,pupper,puppo
0,892420643555336193,2017-08-01 16:23:56,This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,https://twitter.com/dog_rates/status/892420643555336193/photo/1,Phineas,None,None,None,None
1,892177421306343426,2017-08-01 00:17:27,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",https://twitter.com/dog_rates/status/892177421306343426/photo/1,Tilly,None,None,None,None
2,891815181378084864,2017-07-31 00:18:03,This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,https://twitter.com/dog_rates/status/891815181378084864/photo/1,Archie,None,None,None,None
3,891689557279858688,2017-07-30 15:58:51,This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,https://twitter.com/dog_rates/status/891689557279858688/photo/1,Darla,None,None,None,None
4,891327558926688256,2017-07-29 16:00:24,"This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f","https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",Franklin,None,None,None,None
...,...,...,...,...,...,...,...,...,...
2351,666049248165822465,2015-11-16 00:24:50,Here we have a 1949 1st generation vulpix. Enjoys sweat tea and Fox News. Cannot be phased. 5/10 https://t.co/4B7cOc1EDq,https://twitter.com/dog_rates/status/666049248165822465/photo/1,NaN,None,None,None,None
2352,666044226329800704,2015-11-16 00:04:52,This is a purebred Piers Morgan. Loves to Netflix and chill. Always looks like he forgot to unplug the iron. 6/10 https://t.co/DWnyCjf2mx,https://twitter.com/dog_rates/status/666044226329800704/photo/1,NaN,None,None,None,None
2353,666033412701032449,2015-11-15 23:21:54,Here is a very happy pup. Big fan of well-maintained decks. Just look at that tongue. 9/10 would cuddle af https://t.co/y671yMhoiR,https://twitter.com/dog_rates/status/666033412701032449/photo/1,NaN,None,None,None,None
2354,666029285002620928,2015-11-15 23:05:30,This is a western brown Mitsubishi terrier. Upset about leaf. Actually 2 dogs here. 7/10 would walk the shit out of https://t.co/r7mOb2m0UI,https://twitter.com/dog_rates/status/666029285002620928/photo/1,NaN,None,None,None,None


> slash off http in text

In [670]:
adf['text']=adf['text'].str.split('http').str[0]

> re extract dog stages

In [691]:
adf['doggo']=adf['text'].str.extract('([Dd]oggo)')
adf[adf['doggo']=='Doggo']

,tweet_id,timestamp,text,expanded_urls,name,doggo,floofer,pupper,puppo
448,819006400881917954,2017-01-11 02:21:57,This is Sunny. She was also a very good First Doggo. 14/10 would also be an absolute honor to pet,"https://twitter.com/dog_rates/status/819006400881917954/photo/1,https://twitter.com/dog_rates/status/819006400881917954/photo/1,https://twitter.com/dog_rates/status/819006400881917954/photo/1,https://twitter.com/dog_rates/status/819006400881917954/photo/1",Sunny,Doggo,None,None,None
449,819004803107983360,2017-01-11 02:15:36,This is Bo. He was a very good First Doggo. 14/10 would be an absolute honor to pet,"https://twitter.com/dog_rates/status/819004803107983360/photo/1,https://twitter.com/dog_rates/status/819004803107983360/photo/1,https://twitter.com/dog_rates/status/819004803107983360/photo/1,https://twitter.com/dog_rates/status/819004803107983360/photo/1",Bo,Doggo,None,None,None
733,781308096455073793,2016-09-29 01:42:20,"Pupper butt 1, Doggo 0. Both 12/10",https://vine.co/v/5rgu2Law2ut,NaN,Doggo,None,pupper,None
807,771908950375665664,2016-09-03 03:13:29,Doggo will persevere. 13/10\n,https://twitter.com/yahoonews/status/771905568600719360,NaN,Doggo,None,None,None
881,760521673607086080,2016-08-02 17:04:31,"Doggo want what doggo cannot have. Temptation strong, dog stronger. 12/10",https://vine.co/v/5ApKetxzmTB,NaN,Doggo,None,None,None
945,752660715232722944,2016-07-12 00:27:52,Hooman used Pokeball\n*wiggle*\n*wiggle*\nDoggo broke free \n10/10,"https://twitter.com/dog_rates/status/752660715232722944/photo/1,https://twitter.com/dog_rates/status/752660715232722944/photo/1",NaN,Doggo,None,None,None
989,748932637671223296,2016-07-01 17:33:49,Say hello to Divine Doggo. Must be magical af. 13/10 would be an honor to pet,https://twitter.com/dog_rates/status/748932637671223296/photo/1,Divine,Doggo,None,None,None
